In [0]:
display(dbutils.fs.ls("/mnt/bronze-layer/2025-05-12"))

In [0]:
from datetime import datetime, timedelta
from pyspark.sql.functions import col, explode


In [0]:
dbutils.widgets.text("date_val", "")
date_val = dbutils.widgets.get("date_val")

In [0]:
if not date_val:
  yesterday_date = datetime.now() - timedelta(days=1)
  date_val = yesterday_date.strftime("%Y-%m-%d")
date_val

In [0]:
input_path = f'/mnt/bronze-layer/{date_val}/*.json'

df = spark.read.json(input_path)
df.printSchema()


In [0]:
df.show()

In [0]:

df = df.withColumn('fearures', explode(col('features'))).select('fearures.properties')
display(df)

